In [1]:
import pandas as pd
from functions.openai_functions import OpenAI_Client
from functions.db_functions import MySQL, PostGreSQL

In [2]:
openai = OpenAI_Client()
mysql = MySQL()
postgres = PostGreSQL()

In [3]:
df = mysql.load_chunk(chunk_size = 20)

In [4]:
embeddings = [openai.get_embedding(df.loc[i, 'string_for_embedding']) for i in range(len(df))]

In [5]:
df.head()

,id,round,clue_value,daily_double_value,category,comments,answer,question,air_date,notes,string_for_embedding,created_at
0,1,1,100,0,LAKES & RIVERS,,River mentioned most often in the Bible,the Jordan,1984-09-10,,"In round 1, with a value of 100, a daily doubl...",2025-05-01 18:17:12
1,2,1,200,0,LAKES & RIVERS,,Scottish word for lake,loch,1984-09-10,,"In round 1, with a value of 200, a daily doubl...",2025-05-01 18:17:12
2,3,1,400,0,LAKES & RIVERS,,American river only 33 miles shorter than the ...,the Missouri,1984-09-10,,"In round 1, with a value of 400, a daily doubl...",2025-05-01 18:17:12
3,4,1,500,0,LAKES & RIVERS,,"Worlds largest lake, nearly 5 times as big as ...",the Caspian Sea,1984-09-10,,"In round 1, with a value of 500, a daily doubl...",2025-05-01 18:17:12
4,5,1,100,0,INVENTIONS,,Marconis wonderful wireless,a radio,1984-09-10,,"In round 1, with a value of 100, a daily doubl...",2025-05-01 18:17:12


In [6]:
df1 = pd.DataFrame({'id' : df['id'], 'content': df['string_for_embedding'], 'embedding': embeddings})

In [7]:
postgres.insert_data(df1, table_name = 'documents')

Inserted 20 rows into documents


In [64]:
prompt = "What jeopardy questions have been focused around technology, specifically that might use electromagnetic waves?"
embedding = openai.get_embedding(prompt)
prompt_vector = '[' + ','.join(map(str, embedding)) + ']'

In [65]:
import psycopg2
conn = psycopg2.connect(f"host={postgres.host} dbname={postgres.db_name} user={postgres.user} password={postgres.password}")
cursor = conn.cursor()

In [66]:
match_threshold = 0.7
match_count = 5

In [68]:
cursor = conn.cursor()
cursor.execute(
    'SELECT id, content '
    'FROM documents '
    'ORDER BY embedding <=> %(prompt_vector)s LIMIT %(match_cnt)s',
    {'prompt_vector': prompt_vector, 'match_threshold': match_threshold, 'match_cnt': match_count}
    )
result = cursor.fetchall()
cursor.close()
for row in result:
    print(row)

(5, 'In round 1, with a value of 100, a daily double value of 0, in the category INVENTIONS, the question was Marconis wonderful wireless and the answer was a radio. This was on 1984-09-10.')
(7, 'In round 1, with a value of 300, a daily double value of 0, in the category INVENTIONS, the question was A 1920s hunting trip to Canada inspired Birdseyes food preserving method and the answer was freezing. This was on 1984-09-10.')
(8, 'In round 1, with a value of 400, a daily double value of 0, in the category INVENTIONS, the question was This fastener gets its name from a brand of galoshes it was used on and the answer was a zipper. This was on 1984-09-10.')
(15, 'In round 1, with a value of 200, a daily double value of 0, in the category FOREIGN CUISINE, the question was A British variety is called bangers, a Mexican variety, chorizo and the answer was sausages. This was on 1984-09-10.')
(16, 'In round 1, with a value of 300, a daily double value of 0, in the category FOREIGN CUISINE, the

In [ ]:

for row in result:
    print(row)

ProgrammingError: no results to fetch

In [63]:
cursor.close()

In [58]:
for partition in result.partitions():
    # partition is an iterable that will be at most 100 items
    for row in partition:
        print(f"{row}")

(1, '[1,2,3]')
(2, '[4,5,6]')


In [ ]:
query = ""

'In round 1, with a value of 200, a daily double value of 0, in the category LAKES & RIVERS, the question was Scottish word for lake and the answer was loch. This was on 1984-09-10.'

In [5]:
postgres.get_last_id(table_name = 'jeopardy_embeddings')

0

In [ ]:
mysql.load_from_last_id(last_id = 354, chunk_size = 20)